In [2]:
from collections import deque

In [3]:
# Define keypads
numeric_keypad = [
    ["7", "8", "9"],
    ["4", "5", "6"],
    ["1", "2", "3"],
    [" ", "0", "A"]
]

control_keypad = [
    [" ", "^", "A"],
    ["<", "v", ">"]
]

In [4]:
# Input codes
inputs = [
"459A",
"671A",
"846A",
"285A",
"083A"
]

In [5]:
# Number of control keypads
levels = 3

## Part 1

In [6]:
def build_keypad_grid(keypad):
    """Create a grid representation of the keypad."""
    grid = {}
    for r, row in enumerate(keypad):
        for c, key in enumerate(row):
            if key != " ":  # Skip gaps
                grid[key] = (r, c)
    return grid

In [7]:
def bfs_shortest_path(start, target, grid):
    """Find the shortest path between two keys on the grid."""
    directions = {
        'U': (-1, 0),
        'D': (1, 0),
        'L': (0, -1),
        'R': (0, 1)
    }
    reverse_dir = {v: k for k, v in directions.items()}
    
    queue = deque([(start, [])])  # (current position, path to position)
    visited = set()

    while queue:
        current, path = queue.popleft()

        if current == target:
            return path

        if current in visited:
            continue
        visited.add(current)

        for d, (dr, dc) in directions.items():
            next_pos = (current[0] + dr, current[1] + dc)
            if next_pos in grid.values():
                queue.append((next_pos, path + [reverse_dir[(dr, dc)]]))

In [17]:
def calculate_code_complexity(code, numeric_keypad, control_keypad, levels):
    """Recursively calculate complexity for typing a single code."""
    numeric_grid = build_keypad_grid(numeric_keypad)
    current_numeric_pos = numeric_grid["A"]  # Numeric keypad starts on 'A'

    # Generate sequence to press buttons on the numeric keypad
    numeric_sequence = []
    for digit in code:
        target_pos = numeric_grid[digit]
        path = bfs_shortest_path(current_numeric_pos, target_pos, numeric_grid)
        numeric_sequence.extend(path)
        numeric_sequence.append("A")  # Press the button
        current_numeric_pos = target_pos

    # Simulate the control keypads recursively
    total_steps = 0
    for _ in range(levels):
        control_grid = build_keypad_grid(control_keypad)
        direction_map = {'U': '^', 'D': 'v', 'L': '<', 'R': '>'}  # Map movements to control keypad buttons
        current_pos = control_grid["A"]
        total_steps_for_level = 0

        for target in numeric_sequence:
            if target != "A":
                target = direction_map[target]
            if target not in control_grid and target == "A":
                total_steps_for_level += 1
                continue
            target_pos = control_grid[target]
            path = bfs_shortest_path(current_pos, target_pos, control_grid)
            total_steps_for_level += len(path) + 1  # +1 for 'A' to press button
            current_pos = target_pos
            
        numeric_sequence = ["A"] * total_steps  # Sequence for the parent level
            
        total_steps += total_steps_for_level

    # Final complexity
    return total_steps * int(code[:-1].lstrip("0"))

In [18]:
sum(calculate_code_complexity(code, numeric_keypad, control_keypad, levels) for code in inputs)

142552

In [25]:
from collections import deque

def build_keypad_grid(keypad):
    """Create a grid representation of the keypad."""
    grid = {}
    for r, row in enumerate(keypad):
        for c, key in enumerate(row):
            if key != " ":  # Skip gaps
                grid[key] = (r, c)
    return grid

def bfs_shortest_path(start, target, grid):
    """Find the shortest path between two keys on the grid."""
    directions = {
        'U': (-1, 0),
        'D': (1, 0),
        'L': (0, -1),
        'R': (0, 1)
    }
    reverse_dir = {v: k for k, v in directions.items()}

    queue = deque([(start, [])])  # (current position, path to position)
    visited = set()

    while queue:
        current, path = queue.popleft()

        if current == target:
            return path

        if current in visited:
            continue
        visited.add(current)

        for d, (dr, dc) in directions.items():
            next_pos = (current[0] + dr, current[1] + dc)
            if next_pos in grid.values():
                queue.append((next_pos, path + [d]))

    return []  # Should not occur if target is reachable

def simulate_keypad_control(start_pos, target_seq, control_keypad):
    """Simulate a control keypad directing a lower-level keypad."""
    control_grid = build_keypad_grid(control_keypad)
    direction_map = {'U': '^', 'D': 'v', 'L': '<', 'R': '>'}  # Map movements to control keypad buttons
    current_pos = control_grid[start_pos]
    full_sequence = []

    for target in target_seq:
        if target in direction_map:  # Translate movement directions to control keypad buttons
            target = direction_map[target]
        if target not in control_grid:
            if target == 'A':  # Handle button press explicitly
                full_sequence.append('A')
                continue
            raise KeyError(f"Invalid target '{target}' for the control keypad.")
        target_pos = control_grid[target]
        path = bfs_shortest_path(current_pos, target_pos, control_grid)
        full_sequence.extend(path)
        full_sequence.append('A')  # Add the button press
        current_pos = target_pos

    return full_sequence

def calculate_code_complexity(code, numeric_keypad, control_keypad, levels):
    """Recursively calculate complexity for typing a single code."""
    numeric_grid = build_keypad_grid(numeric_keypad)
    current_numeric_pos = numeric_grid['A']  # Numeric keypad starts on 'A'

    # Generate sequence to press buttons on the numeric keypad
    numeric_sequence = []
    for digit in code:
        target_pos = numeric_grid[digit]
        path = bfs_shortest_path(current_numeric_pos, target_pos, numeric_grid)
        numeric_sequence.extend(path)
        numeric_sequence.append('A')  # Press the button
        current_numeric_pos = target_pos

    # Simulate the control keypads recursively
    for _ in range(levels):
        numeric_sequence = simulate_keypad_control('A', numeric_sequence, control_keypad)

    # Final complexity
    total_steps = len(numeric_sequence)
    return total_steps * int(code[:-1].lstrip("0"))

# Define keypads
numeric_keypad = [
    ["7", "8", "9"],
    ["4", "5", "6"],
    ["1", "2", "3"],
    [" ", "0", "A"]
]

control_keypad = [
    [" ", "^", "A"],
    ["<", "v", ">"]
]

# Input codes
codes = ["029A", "980A", "179A", "456A", "379A"]

# Number of control keypads
levels = 3

# Calculate total complexity
total_complexity = sum(
    calculate_code_complexity(code, numeric_keypad, control_keypad, levels) for code in codes
)

print("Total Complexity:", total_complexity)


Total Complexity: 346762


In [27]:
126384/total_complexity

0.3644690017937375

In [29]:
total_complexity/126384

2.7437175591847067